In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/Image-Captioning-master

/content/drive/MyDrive/Image-Captioning-master


In [4]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [7]:
!python CapGenerator/prepare_data.py

python3: can't open file 'CapGenerator/prepare_data.py': [Errno 2] No such file or directory


In [6]:
!python CapGenerator/train_model.py

python3: can't open file 'CapGenerator/train_model.py': [Errno 2] No such file or directory


In [ ]:
!python CapGenerator/eval_modelMultiImage.py

Using TensorFlow backend.
models/wholeModel.h5
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
2021-03-24 18:20:23.605598: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-03-24 18:20:23.633972: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-03-24 18:20:23.634622: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Found device 0 with properties: 
name: Tesla V100-SXM2-16GB major: 7 minor: 0 memoryClockRate(GHz): 1.53
pciBusID: 0000:00:04.0
2021-03-24 18:20:23.634921: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2021-03-24 18:20:23.636652: I tensorflow/stream_executor/p

In [ ]:
%cd /content/drive/MyDrive/googleDataset

/content/drive/MyDrive/googleDataset


In [ ]:
import os
import PIL.Image
import urllib.request
import pandas as pd
import requests
import shutil
import csv

def resize_and_crop(img, size):


    # Get current and desired ratio for the images
    img_ratio = img.size[0] / float(img.size[1])
    ratio = size[0] / float(size[1])
    #The image is scaled/cropped vertically or horizontally depending on the ratio
    if ratio > img_ratio:
        img = img.resize((size[0], int(size[0] * img.size[1] / img.size[0])), PIL.Image.ANTIALIAS)
        box = (0, (img.size[1] - size[1]) / 2, img.size[0], (img.size[1] + size[1]) / 2)
        img = img.crop(box)
    elif ratio < img_ratio:
        img = img.resize((int(size[1] * img.size[0] / img.size[1]), size[1]), PIL.Image.ANTIALIAS)
        box = ((img.size[0] - size[0]) / 2, 0, (img.size[0] + size[0]) / 2, img.size[1])
        img = img.crop(box)
    else :
        img = img.resize((size[0], size[1]),PIL.Image.ANTIALIAS)
        # If the scale is the same, we do not need to crop
    return img

def main():
    DIR = 'formatted'
    data = pd.read_table("Train_GCC-training.tsv", usecols=[0,1], names=['caption', 'url'], header=None)
    txtFile = open('imageCaptions.txt', 'w', newline='')
    #print(data)
    for i in range(20000):
        t = "{:05d}".format(i)
        fName = 'gImg_' + t + '.png'
        cap = data['caption'][i]
        url = data['url'][i]

        #urllib.request.urlretrieve(url,fName)
        try:            
            image = PIL.Image.open(requests.get(url, stream=True, timeout=10).raw)
            image = resize_and_crop(image, [244, 244])
            if image.mode != 'RGB':
                image = image.convert('RGB')
            image.save(DIR + '/' + fName)
            txtFile.write(fName + ',' + cap)
            txtFile.write('\n')
            
    
        except:

            continue




    txtFile.close()





if __name__ == "__main__":
    main()

In [ ]:
!apt install -qq enchant
!pip install pyenchant

The following additional packages will be installed:
  aspell aspell-en dictionaries-common emacsen-common hunspell-en-us
  libaspell15 libenchant1c2a libhunspell-1.6-0 libtext-iconv-perl
Suggested packages:
  aspell-doc spellutils wordlist hunspell openoffice.org-hunspell
  | openoffice.org-core libenchant-voikko
The following NEW packages will be installed:
  aspell aspell-en dictionaries-common emacsen-common enchant hunspell-en-us
  libaspell15 libenchant1c2a libhunspell-1.6-0 libtext-iconv-perl
0 upgraded, 10 newly installed, 0 to remove and 30 not upgraded.
Need to get 1,310 kB of archives.
After this operation, 5,353 kB of additional disk space will be used.
Preconfiguring packages ...
Selecting previously unselected package libtext-iconv-perl.
(Reading database ... 160980 files and directories currently installed.)
Preparing to unpack .../0-libtext-iconv-perl_1.7-5build6_amd64.deb ...
Unpacking libtext-iconv-perl (1.7-5build6) ...
Selecting previously unselected package libaspe

In [ ]:
import enchant
import string
import random
import shutil
import os

def main():
    en_us = enchant.Dict("en_US")
    txtFile = open('imageCaptions.txt', 'r')
    trainFile = open('trainImages.txt', 'w')
    devFile = open('devImages.txt', 'w')
    fixFile = open('fixedCaptions.txt', 'w')
    lines = txtFile.readlines()
    DIR= 'Flickr8k_Dataset'
    SDIR = 'target'
    fList = []
    for i in range(0,10000):
        row = lines[i].split(',')
        if len(row)>1:
            if '.png' in row[0]:
                fList.append(row[0])
                if len(row) == 2:
                    newStr = row[1]
                    tempStr = newStr.split()
                    newStr = ""
                    for j in range(len(tempStr)):
                        if j < 10:
                            newStr = newStr + ' ' + tempStr[j]
                    newStr = ' '.join(w for w in newStr.split() if en_us.check(w))
                    newStr = newStr.replace('\n','')
                    exclude = set(string.punctuation)
                    newStr = ''.join(ch for ch in newStr if ch not in exclude)
                    fixFile.write(row[0] + ' ' + newStr)
                    if random.random() < .2:
                        devFile.write(row[0])
                        if i < len(lines)-1:
                            devFile.write('\n')
                    else:
                        trainFile.write(row[0])
                        if i < len(lines)-1:
                            trainFile.write('\n')

                else:
                    newStr = ""
                    for j in range(1,len(row)):
                        newStr = newStr + row[j]
                    tempStr = newStr.split()
                    newStr = ""
                    for j in range(len(tempStr)):
                        if j < 10:
                            newStr = newStr + ' ' + tempStr[j]
                    newStr = ' '.join(w for w in newStr.split() if en_us.check(w))
                    newStr = newStr.replace('\n','')
                    exclude = set(string.punctuation)
                    newStr = ''.join(ch for ch in newStr if ch not in exclude)
                    fixFile.write(row[0] + ' ' + newStr)


                if i < len(lines)-1:
                    fixFile.write('\n')
    
    for i in range(10000,len(lines)):
        row = lines[i].split(',')
        if '.png' in row[0]:
            source = DIR+'/'+row[0]
            try:
                os.remove(source)
                print(row[0])
            except:
                continue

    fList = list(set(fList))
    for i in range(len(fList)):
        if random.random() < .9:
            trainFile.write(fList[i])
            trainFile.write('\n')
        else:
            devFile.write(fList[i])
            devFile.write('\n')

    fixFile.close()
    trainFile.close()
    devFile.close()
    txtFile.close()

if __name__ == "__main__":
    main()

IndexError: ignored